# Election Results 2023

Downloading the election results from:

https://results.eci.gov.in/AcResultGenDecNew2023/

In [9]:
import pandas as pd
import json
import requests
import functools
from pathlib import Path

## Download Data

Data Organization:
    
```
data/
    states.csv
    results.csv
    constituencies.csv
    S12/
        features.json
        constituencies.csv
        001.csv
        002.csv
```

In [3]:
# Chhattisgarh, Madhya Pradesh, Rajasthan and Telangana
state_codes = ["S12", "S20", "S26", "S29"]

## The constituencies

The ECI website seems to have a JS file that it is using for rendering the map containing all the boundaries of the map and constituency details of every constituency in a state. The following code extracts the ac code and names from it.

In [30]:
@functools.cache
def get_features(st_code):
    path = Path("data") / st_code / "features.json"
    if path.exists():
        jsontext = path.read_text()
    else:
        path.parent.mkdir(exist_ok=True, parents=True)

        url = f"https://results.eci.gov.in/AcResultGenDecNew2023/ac/{st_code}.js"
        jsontext = requests.get(url).text.split("=")[1]
        path.write_text(jsontext)
        print("saved", path)
    return json.loads(jsontext)
    
@functools.cache
def get_constituencies(st_code):
    """Returns constituencies of the state as a dataframe.
    """
    path = Path("data") / st_code / "constituencies.csv"
    if path.exists():
        return pd.read_csv(path)
        
    features = get_features(st_code) 
    constituencies = [row['properties'] for row in features['features']]
    constituencies = sorted(constituencies, key=lambda ac: ac['AC_NO'])
    df = pd.DataFrame(constituencies)
    df = df.drop(columns=['FID'])
    df.columns = [c.lower() for c in df.columns]
    df.to_csv(path, index=False)
    print("saved", path)
    return df

In [32]:
for code in state_codes:
    get_constituencies(code)

saved data/S12/features.json
saved data/S12/constituencies.csv
saved data/S20/features.json
saved data/S20/constituencies.csv
saved data/S26/features.json
saved data/S26/constituencies.csv
saved data/S29/features.json
saved data/S29/constituencies.csv


In [75]:
dfs = [get_constituencies(code) for code in state_codes]

In [76]:
df = pd.concat(dfs)

In [77]:
df.head()

,ac_no,st_code,st_name,ac_name
0,1,S12,Madhya Pradesh,Sheopur
1,2,S12,Madhya Pradesh,Vijaypur
2,3,S12,Madhya Pradesh,Sabalagadh
3,4,S12,Madhya Pradesh,Jaura
4,5,S12,Madhya Pradesh,Sumaoli


In [78]:
df.to_csv("data/constituencies.csv", index=False)

In [79]:
df.head()

,ac_no,st_code,st_name,ac_name
0,1,S12,Madhya Pradesh,Sheopur
1,2,S12,Madhya Pradesh,Vijaypur
2,3,S12,Madhya Pradesh,Sabalagadh
3,4,S12,Madhya Pradesh,Jaura
4,5,S12,Madhya Pradesh,Sumaoli


In [91]:
df_states = (
    df
    .groupby(['st_code', 'st_name'])
    .count()
    [['ac_no']]
    .rename(columns={"ac_no": "num_constituencies"})
    .reset_index()
)

In [92]:
df_states

,st_code,st_name,num_constituencies
0,S12,Madhya Pradesh,230
1,S20,Rajasthan,200
2,S26,Chhattisgarh,90
3,S29,Telangana,119


In [93]:
df_states.to_csv("data/states.csv")

In [40]:
df1 = get_constituencies("S12")

In [42]:
df1.head()

,ac_no,st_code,st_name,ac_name
0,1,S12,Madhya Pradesh,Sheopur
1,2,S12,Madhya Pradesh,Vijaypur
2,3,S12,Madhya Pradesh,Sabalagadh
3,4,S12,Madhya Pradesh,Jaura
4,5,S12,Madhya Pradesh,Sumaoli


In [44]:
df1.set_index('ac_no').loc[1]

st_code               S12
st_name    Madhya Pradesh
ac_name           Sheopur
Name: 1, dtype: object

## Download Results

Code to download results of each constituency.

In [59]:

def read_table(st_code, ac_no):
    url = f"https://results.eci.gov.in/AcResultGenDecNew2023/Constituencywise{st_code}{ac_no}.htm"
    print("Parsing", url)
    dfs = pd.read_html(url)
    return preprocess_data(dfs[0])

def preprocess_data(df):
    """Cleans up the data to remove unwanted rows, renames the columns.
    """
    columns = {
        "Candidate": "candidate",
        "Party": "party",
        "EVM Votes": "evm_votes",
        "Postal Votes": "postal_votes",
        "Total Votes": "total_votes",
        "% of Votes": "percent_votes"
    }
    return (
        df
        .iloc[:-1] # skip last row
        .drop(columns=["S.N."])
        .rename(columns=columns))

def download_results(st_code, ac_no):
    path = Path("data") / st_code / f"{ac_no:03d}.csv"
    path.parent.mkdir(parents=True, exist_ok=True)
    if path.exists():
        print(f"{path} already exists. reused...")
        return pd.read_csv(path)
    
    try:
        df = read_table(st_code, ac_no)
    except Exception: 
        print(f"Failed to download results for {st_code} {ac_no}")
        import traceback
        traceback.print_exc()
        return None
    df1 = get_constituencies("S12")   
    row = df1.set_index('ac_no').loc[ac_no]
    
    df['st_code'] = row.st_code
    df['st_name'] = row.st_name
    df['ac_no'] = ac_no
    df['ac_name'] = row.ac_name
    
    # reorder columns
    columns = list(df.columns[-4:]) + list(df.columns[:-4])
    df = df[columns]
    
    df.to_csv(path, index=False)
    print("saved", path)
    return df

def download_state(code):
    path = Path("data") / code / f"results.csv"
    if path.exists():
        return pd.read_csv(path)

    df1 = get_constituencies(code)
    dfs = [download_results(code, ac_no) for ac_no in df1.ac_no]
    dfs = [df for df in dfs if df is not None]
    df = pd.concat(dfs)
    df.to_csv(path, index=False)
    return df


In [60]:
for code in state_codes:
    download_state(code)

data/S20/001.csv already exists. reused...
data/S20/002.csv already exists. reused...
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS203.htm
Failed to download results for S20 3
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS204.htm
saved data/S20/004.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS205.htm


Traceback (most recent call last):
  File "/tmp/ipykernel_129361/1467209966.py", line 32, in download_results
    df = read_table(state_code, ac_no)
  File "/tmp/ipykernel_129361/1467209966.py", line 4, in read_table
    dfs = pd.read_html(url)
  File "/home/anand/.local/lib/python3.10/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "/home/anand/.local/lib/python3.10/site-packages/pandas/io/html.py", line 1205, in read_html
    return _parse(
  File "/home/anand/.local/lib/python3.10/site-packages/pandas/io/html.py", line 986, in _parse
    tables = p.parse_tables()
  File "/home/anand/.local/lib/python3.10/site-packages/pandas/io/html.py", line 262, in parse_tables
    tables = self._parse_tables(self._build_doc(), self.match, self.attrs)
  File "/home/anand/.local/lib/python3.10/site-packages/pandas/io/html.py", line 821, in _build_doc
    raise e
  File "/home/anand/.local/lib/python3.10/site-packages/pandas/io/html.py", line 8

saved data/S20/005.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS206.htm
saved data/S20/006.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS207.htm
saved data/S20/007.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS208.htm
saved data/S20/008.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS209.htm
saved data/S20/009.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS2010.htm
saved data/S20/010.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS2011.htm
saved data/S20/011.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS2012.htm
saved data/S20/012.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS2013.htm
saved data/S20/013.csv
Parsing https://results.eci.gov.in/AcResultGenDecNew2023/ConstituencywiseS2014.htm
saved data/S20/014.csv
Parsing https://results.eci

## All Results

In [71]:
dfs = [download_state(code) for code in state_codes]

In [72]:
df = pd.concat(dfs)

In [73]:
df.head()

,st_code,st_name,ac_no,ac_name,candidate,party,evm_votes,postal_votes,total_votes,percent_votes
0,S12,Madhya Pradesh,1,Sheopur,DURGALAL VIJAY (VAKEEL SAAB),Bharatiya Janata Party,85006,708,85714,40.45
1,S12,Madhya Pradesh,1,Sheopur,BABU JANDEL,Indian National Congress,95602,1242,96844,45.70
2,S12,Madhya Pradesh,1,Sheopur,BIHARI SINGH SOLANKI,Bahujan Samaj Party,22833,221,23054,10.88
3,S12,Madhya Pradesh,1,Sheopur,ADIL KHAN,Social Democratic Party Of India,729,4,733,0.35
4,S12,Madhya Pradesh,1,Sheopur,YOGESH TYAGI,Rashtriya Krantikari Samajwadi Party,233,1,234,0.11


In [74]:
df.to_csv("data/results.csv", index=False)